In [51]:
import torch


torch.manual_seed(0)

A = torch.randint(-10, 10,(4,6))
B = torch.randint(-10, 10,(2, 5,6))

A, B

(tensor([[ -6,   9,   3, -10,  -7,   9],
         [ -3,  -7,   7,  -7,  -9,  -4],
         [  6,   9,   8,   6,   6,  -2],
         [  4,   3,  -4,   9,   1,   4]]),
 tensor([[[ -6,  -9,  -1,  -1,  -1, -10],
          [ -9,   2,  -7, -10,   5,  -5],
          [ -8,   9,   1,  -2,   8,  -7],
          [  6,  -1,   1,   7,  -7,   5],
          [ -8,   1, -10,  -1,   3,  -9]],
 
         [[  1, -10,  -7,  -4,   6,  -3],
          [  9,  -4,  -7,   4,   5, -10],
          [  8,   2,  -2,   2,   7,  -5],
          [  0, -10,  -2,  -9,   9,   6],
          [ -9,   0,  -8,   9,  -6,   3]]]))

In [52]:
A_exp = A.unsqueeze(0).expand(B.shape[1], -1, -1).unsqueeze(0).expand(B.shape[0],-1,-1,-1)
B_exp = B.unsqueeze(2).expand(-1, -1, A.shape[0], -1)

print(A_exp.shape, B_exp.shape)
A_exp, B_exp

torch.Size([2, 5, 4, 6]) torch.Size([2, 5, 4, 6])


(tensor([[[[ -6,   9,   3, -10,  -7,   9],
           [ -3,  -7,   7,  -7,  -9,  -4],
           [  6,   9,   8,   6,   6,  -2],
           [  4,   3,  -4,   9,   1,   4]],
 
          [[ -6,   9,   3, -10,  -7,   9],
           [ -3,  -7,   7,  -7,  -9,  -4],
           [  6,   9,   8,   6,   6,  -2],
           [  4,   3,  -4,   9,   1,   4]],
 
          [[ -6,   9,   3, -10,  -7,   9],
           [ -3,  -7,   7,  -7,  -9,  -4],
           [  6,   9,   8,   6,   6,  -2],
           [  4,   3,  -4,   9,   1,   4]],
 
          [[ -6,   9,   3, -10,  -7,   9],
           [ -3,  -7,   7,  -7,  -9,  -4],
           [  6,   9,   8,   6,   6,  -2],
           [  4,   3,  -4,   9,   1,   4]],
 
          [[ -6,   9,   3, -10,  -7,   9],
           [ -3,  -7,   7,  -7,  -9,  -4],
           [  6,   9,   8,   6,   6,  -2],
           [  4,   3,  -4,   9,   1,   4]]],
 
 
         [[[ -6,   9,   3, -10,  -7,   9],
           [ -3,  -7,   7,  -7,  -9,  -4],
           [  6,   9,   8,   6,   6,

In [53]:
squared_diffs = (A_exp - B_exp) ** 2
squared_diffs

tensor([[[[  0, 324,  16,  81,  36, 361],
          [  9,   4,  64,  36,  64,  36],
          [144, 324,  81,  49,  49,  64],
          [100, 144,   9, 100,   4, 196]],

         [[  9,  49, 100,   0, 144, 196],
          [ 36,  81, 196,   9, 196,   1],
          [225,  49, 225, 256,   1,   9],
          [169,   1,   9, 361,  16,  81]],

         [[  4,   0,   4,  64, 225, 256],
          [ 25, 256,  36,  25, 289,   9],
          [196,   0,  49,  64,   4,  25],
          [144,  36,  25, 121,  49, 121]],

         [[144, 100,   4, 289,   0,  16],
          [ 81,  36,  36, 196,   4,  81],
          [  0, 100,  49,   1, 169,  49],
          [  4,  16,  25,   4,  64,   1]],

         [[  4,  64, 169,  81, 100, 324],
          [ 25,  64, 289,  36, 144,  25],
          [196,  64, 324,  49,   9,  49],
          [144,   4,  36, 100,   4, 169]]],


        [[[ 49, 361, 100,  36, 169, 144],
          [ 16,   9, 196,   9, 225,   1],
          [ 25, 361, 225, 100,   0,   1],
          [  9, 169,  

In [54]:
summm = squared_diffs.sum(dim=3)
summm

tensor([[[ 818,  213,  711,  553],
         [ 498,  519,  765,  637],
         [ 553,  640,  338,  496],
         [ 553,  434,  368,  114],
         [ 742,  583,  691,  457]],

        [[ 859,  456,  712,  430],
         [1195,  702,  472,  320],
         [ 806,  621,  179,  187],
         [ 688,  527,  795,  581],
         [ 609,  624,  740,  244]]])

In [55]:
summm = squared_diffs.mean(dim=3, dtype=torch.float32)
summm

tensor([[[136.3333,  35.5000, 118.5000,  92.1667],
         [ 83.0000,  86.5000, 127.5000, 106.1667],
         [ 92.1667, 106.6667,  56.3333,  82.6667],
         [ 92.1667,  72.3333,  61.3333,  19.0000],
         [123.6667,  97.1667, 115.1667,  76.1667]],

        [[143.1667,  76.0000, 118.6667,  71.6667],
         [199.1667, 117.0000,  78.6667,  53.3333],
         [134.3333, 103.5000,  29.8333,  31.1667],
         [114.6667,  87.8333, 132.5000,  96.8333],
         [101.5000, 104.0000, 123.3333,  40.6667]]])

In [63]:
summm.min(dim=2).indices

tensor([[1, 0, 2, 3, 3],
        [3, 3, 2, 1, 3]])

In [66]:
real_val = torch.tensor([[1,0,0,0,0], [3,0,0,0,3]])

(summm.min(dim=2).indices != real_val).sum()


tensor(6)

In [57]:
y = torch.tensor([[0,1], [2,3]])
y

tensor([[0, 1],
        [2, 3]])

In [58]:
indices_unsqueezed = y.unsqueeze(2)  # Adds a third dimension for compatibility with gather
selected = torch.gather(aaa, 2, indices_unsqueezed).squeeze(2)

selected


tensor([[136.3333,  86.5000],
        [134.3333,  87.8333]])

In [59]:
selected.view(-1).mean()

tensor(111.2500)

In [60]:
from torch.nn import functional as F
print(aaa)
print(aaa.shape)
print(y)
print(y.shape)

B, T, C = aaa.shape
aaa = aaa.reshape(B*T, C)
print(aaa)
F.cross_entropy(aaa, y.view(-1), reduction='none')

tensor([[[136.3333,  83.0000,  92.1667,  92.1667, 123.6667],
         [ 35.5000,  86.5000, 106.6667,  72.3333,  97.1667],
         [118.5000, 127.5000,  56.3333,  61.3333, 115.1667],
         [ 92.1667, 106.1667,  82.6667,  19.0000,  76.1667]],

        [[143.1667, 199.1667, 134.3333, 114.6667, 101.5000],
         [ 76.0000, 117.0000, 103.5000,  87.8333, 104.0000],
         [118.6667,  78.6667,  29.8333, 132.5000, 123.3333],
         [ 71.6667,  53.3333,  31.1667,  96.8333,  40.6667]]])
torch.Size([2, 4, 5])
tensor([[0, 1],
        [2, 3]])
torch.Size([2, 2])
tensor([[136.3333,  83.0000,  92.1667,  92.1667, 123.6667],
        [ 35.5000,  86.5000, 106.6667,  72.3333,  97.1667],
        [118.5000, 127.5000,  56.3333,  61.3333, 115.1667],
        [ 92.1667, 106.1667,  82.6667,  19.0000,  76.1667],
        [143.1667, 199.1667, 134.3333, 114.6667, 101.5000],
        [ 76.0000, 117.0000, 103.5000,  87.8333, 104.0000],
        [118.6667,  78.6667,  29.8333, 132.5000, 123.3333],
        [ 71.6

ValueError: Expected input batch_size (8) to match target batch_size (4).